In [3]:
! pip install pyedgar

In [4]:
from pyedgar import Filing
from pyedgar import EDGARIndex
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [5]:
file_type = '8-K'

In [6]:
filesave_directory = 'C:/Users/Cheng Jiang/Desktop/Research/Project with Ron/Lawyer CEO and Innovation - with Barbara/Projects/Project 20210210 8K files download/'

In [7]:
# set True for first time downloading and False for future time
all_indices = EDGARIndex(force_download=True)

108it [01:39,  1.09it/s]                     
Done downloading, extracting...
100%|██████████| 6/6 [03:35<00:00, 35.84s/it] 
Done!


In [8]:
database = all_indices['8-K']

In [9]:
print(database)

             cik                                        name   form  \
0             20                    K TRON INTERNATIONAL INC    8-K   
1             20                    K TRON INTERNATIONAL INC    8-K   
2             20                    K TRON INTERNATIONAL INC    8-K   
3             20                    K TRON INTERNATIONAL INC    8-K   
4             20                    K TRON INTERNATIONAL INC  8-K/A   
...          ...                                         ...    ...   
1767663  1837019         Ford Credit Auto Lease Trust 2021-A    8-K   
1767664  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767665  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767666  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767667  1838040  Toyota Auto Receivables 2021-A Owner Trust    8-K   

          filedate             accession  
0       1995-07-10  0000893220-95-000441  
1       2001-10-17  0000893220-01-500767  
2       2002-06-26

In [10]:
# only get files later than 1995
database = database.loc[(database['filedate'] > '1995-01-01')]   

In [11]:
# create the new variable called EDGARURL, which has a format like https://www.sec.gov/Archives/edgar/data/1750/0001104659-17-062143.txt
database['EDGARURL'] = 'https://www.sec.gov/Archives/edgar/data/' + database['cik'].apply(str) + '/' + database['accession'].apply(str) +'.txt'

In [12]:
print(database)

             cik                                        name   form  \
0             20                    K TRON INTERNATIONAL INC    8-K   
1             20                    K TRON INTERNATIONAL INC    8-K   
2             20                    K TRON INTERNATIONAL INC    8-K   
3             20                    K TRON INTERNATIONAL INC    8-K   
4             20                    K TRON INTERNATIONAL INC  8-K/A   
...          ...                                         ...    ...   
1767663  1837019         Ford Credit Auto Lease Trust 2021-A    8-K   
1767664  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767665  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767666  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767667  1838040  Toyota Auto Receivables 2021-A Owner Trust    8-K   

          filedate             accession  \
0       1995-07-10  0000893220-95-000441   
1       2001-10-17  0000893220-01-500767   
2       2002-06

In [13]:
total_entries = database['EDGARURL'].count()
print(total_entries)     # there are totally 1,767,668 8K files

1767668


### iteration through all files

In [20]:
for files in range(1767000, total_entries):
    cik = database.iloc[files, 0]   
    accession = database.iloc[files, 4]   
    url = database.iloc[files, 5] 

    r = requests.get(url)
    raw = r.text

    docs_start = re.compile(r'<DOCUMENT>')
    docs_end = re.compile(r'</DOCUMENT>')
    docs_type = re.compile(r'<TYPE>[^\n]+')

    doc_s = [x.end() for x in docs_start.finditer(raw)]
    doc_e = [x.start() for x in docs_end.finditer(raw)]
    doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

    document = {}
    
    filetype = doc_t[0]
    
    for doc_type, doc_start, doc_end in zip(doc_t, doc_s, doc_e):
        if doc_type == filetype:
            document[filetype] = raw[doc_start:doc_end]
            break

    print(filetype, " : ", url)

    filename = (filesave_directory + str(cik) +'/'+ str(accession) + '.txt')

    item_xcontent = BeautifulSoup(document[filetype], 'lxml')

    content = item_xcontent.get_text()

    filename = (filesave_directory + str(cik) +'/'+ str(accession) + '.txt')
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", encoding='utf-8') as f:
        f.write(str(content))
        
    progress = 100*files/total_entries
    print(progress.round(2), "% completed")
    print("-"*100)

print("done")

8-K  :  https://www.sec.gov/Archives/edgar/data/1820630/0001213900-20-029517.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1820630/0001213900-20-036158.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1820630/0001213900-21-001584.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1820630/0001213900-21-006284.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1820721/0001193125-20-272092.txt
99.96 % completed
----------------------------------------------------------------------------------------------------


8-K  :  https://www.sec.gov/Archives/edgar/data/1821018/0001213900-21-005401.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821075/0001213900-20-041816.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821075/0001213900-20-042517.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821075/0001213900-21-003332.txt
99.96 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821146/0001213900-20-037980.txt
99.96 % completed
----------------------------------------------------------------------------------------------------


8-K  :  https://www.sec.gov/Archives/edgar/data/1821595/0001213900-20-043559.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821595/0001213900-20-044098.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-006323.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K/A  :  https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-007131.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1821606/0001193125-21-008241.txt
99.97 % completed
---------------------------------------------------------------------------------------------------

8-K  :  https://www.sec.gov/Archives/edgar/data/1822309/0001213900-20-039267.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822309/0001213900-20-040265.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822309/0001213900-20-042225.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822309/0001213900-21-002622.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822309/0001213900-21-006744.txt
99.97 % completed
----------------------------------------------------------------------------------------------------


8-K  :  https://www.sec.gov/Archives/edgar/data/1822862/0001140361-21-003425.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822866/0001193125-20-253433.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822866/0001193125-20-253442.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822866/0001193125-20-259264.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1822873/0001140361-20-023779.txt
99.97 % completed
----------------------------------------------------------------------------------------------------


8-K  :  https://www.sec.gov/Archives/edgar/data/1823144/0001104659-20-123676.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823144/0001104659-20-126427.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823144/0001104659-20-127402.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823144/0001104659-20-132645.txt
99.97 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823200/0001193125-20-276975.txt
99.97 % completed
----------------------------------------------------------------------------------------------------


99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823608/0001193125-20-268990.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823608/0001823608-20-000007.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823608/0001823608-21-000002.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823608/0001823608-21-000004.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K/A  :  https://www.sec.gov/Archives/edgar/data/1823608/0001823608-21-000007.tx

99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823857/0001213900-21-006233.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823878/0001104659-20-118601.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823878/0001104659-20-120577.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823878/0001104659-20-123836.txt
99.98 % completed
----------------------------------------------------------------------------------------------------
8-K  :  https://www.sec.gov/Archives/edgar/data/1823878/0001104659-20-134100.txt


KeyboardInterrupt: 